# Reading events from RGTDB

Query to get JSON:

https://rgtdb.com/events/json?search=&offset=0&limit=100

This reads upcoming events from rgtdb.com and converts them into iCAL format and writes to a file for manual import. Then this saves or updates the events in a Google calendar.

## Get data and cache response

In [1]:
import pandas as pd 

import datetime
from datetime import datetime as dt

import io
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Debug level logging turned on")

import requests
from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter

sess = requests.session()
cached_sess = CacheControl(sess, cache = FileCache('.web_cache'), heuristic=ExpiresAfter(hours=1))

try:
    response = cached_sess.get('https://rgtdb.com/events/json?search=&offset=0&limit=200') # Get 200 events. Should be about a week's worth of events
    response.raise_for_status()

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

logger.setLevel(logging.ERROR)

print(dt.now())

DEBUG:root:Debug level logging turned on
DEBUG:cachecontrol.controller:Looking up "https://rgtdb.com/events/json?search=&offset=0&limit=200" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 385100
DEBUG:cachecontrol.controller:Freshness lifetime from expires: 3601
DEBUG:cachecontrol.controller:The cached response is "stale" with no etag, purging
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rgtdb.com:443
DEBUG:urllib3.connectionpool:https://rgtdb.com:443 "GET /events/json?search=&offset=0&limit=200 HTTP/1.1" 200 None
DEBUG:cachecontrol.controller:Updating cache with response from "https://rgtdb.com/events/json?search=&offset=0&limit=200"
DEBUG:cachecontrol.controller:Caching b/c of expires header


2021-04-01 18:37:20.576714


In [2]:
response.json().keys()

dict_keys(['total', 'rows'])

## Convert JSON to Pandas Dataframe

Not necessary, but hey, pretending to be a data scientist feels cool.

In [3]:
df = pd.json_normalize(response.json(), 'rows')
df

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
0,STDistribution - Recovery,04-01 16:30,/events/84086,[groupride],20,27.35 km,161 m,161 m,STD Ride In Uster,/courses/101244,False
1,La classique du Jeudi,04-01 16:45,/events/83671,"[race, ranked]",70,36.38 km,322 m,323 m,Critérium Lisieux,/courses/145173,True
2,Port Talbot Wheelers TT,04-01 17:00,/events/84002,"[race, itt]",5,16.13 km,62 m,58 m,R10/22A TT Course,/courses/148804,False
3,Tour de Waffle,04-01 17:00,/events/83292,[race],3,25.16 km,614 m,614 m,Paterberg,/courses/88,False
4,Abandonment of Hope HC,04-01 18:00,/events/80163,[race],42,32.28 km,564 m,309 m,Abandonment Of Hope,/courses/159393,False
...,...,...,...,...,...,...,...,...,...,...,...
125,WESTERLEY VR RACE 3.10,08-18 18:30,/events/75315,"[race, ranked]",11,15.64 km,72 m,45 m,Aces High WVRace,/courses/144701,True
126,OTR WorldTour - Bretagne,08-22 14:00,/events/83874,"[race, ranked]",1,60.00 km,937 m,999 m,Bretagne Classic,/courses/165863,True
127,WESTERLEY VR RACE 3.11,09-01 19:30,/events/72711,"[race, ranked]",15,16.30 km,37 m,8 m,Young Berks And Old,/courses/137898,True
128,WESTERLEY VR RACE 3.12,09-15 18:30,/events/75316,"[race, ranked]",10,16.38 km,119 m,119 m,The Hillingdon Honey Races,/courses/141054,True


## Convert start time into datetime format - and guess at the year

This will be an issue every year around new year.

In [4]:

this_year = str(dt.today().year)
df['date'] = pd.to_datetime(df['startAt'] + ' ' + this_year, format='%m-%d %H:%M %Y', utc=True)
df.set_index('date', inplace=True)


In [5]:
df.head()

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
date,,,,,,,,,,,
2021-04-01 16:30:00+00:00,STDistribution - Recovery,04-01 16:30,/events/84086,[groupride],20,27.35 km,161 m,161 m,STD Ride In Uster,/courses/101244,False
2021-04-01 16:45:00+00:00,La classique du Jeudi,04-01 16:45,/events/83671,"[race, ranked]",70,36.38 km,322 m,323 m,Critérium Lisieux,/courses/145173,True
2021-04-01 17:00:00+00:00,Port Talbot Wheelers TT,04-01 17:00,/events/84002,"[race, itt]",5,16.13 km,62 m,58 m,R10/22A TT Course,/courses/148804,False
2021-04-01 17:00:00+00:00,Tour de Waffle,04-01 17:00,/events/83292,[race],3,25.16 km,614 m,614 m,Paterberg,/courses/88,False
2021-04-01 18:00:00+00:00,Abandonment of Hope HC,04-01 18:00,/events/80163,[race],42,32.28 km,564 m,309 m,Abandonment Of Hope,/courses/159393,False


## Use icalendar package to create ICAL format events

* GitHub: https://github.com/collective/icalendar


In [6]:
from datetime import timedelta
from icalendar import vCalAddress, vText
from icalendar import Calendar, Event
import pytz

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.com//')
cal.add('version', '2.0')

for index, row in df.iterrows():
    print(index, row['name'], row['tags'])
    event = Event()
    event['uid'] = row['detailsUrl']
    event.add('summary', str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']))
    event.add('dtstart', index)
    event.add('dtend', index  + timedelta(hours=1))
    event.add('url', 'https://rgtdb.com' + row['detailsUrl'])
    event.add('description', row['distance'] + ' ' + 'https://rgtdb.com' + row['detailsUrl'])
    event.add('color', 'Tomato')
    event['location'] = vText(row['roadName'])

    cal.add_component(event)

2021-04-01 16:30:00+00:00 STDistribution - Recovery ['groupride']
2021-04-01 16:45:00+00:00 La classique du Jeudi ['race', 'ranked']
2021-04-01 17:00:00+00:00 Port Talbot Wheelers TT ['race', 'itt']
2021-04-01 17:00:00+00:00 Tour de Waffle ['race']
2021-04-01 18:00:00+00:00 Abandonment of Hope HC  ['race']
2021-04-01 18:00:00+00:00 Tour of Flanders Group Ride with Johan Museeuw ['groupride']
2021-04-01 18:00:00+00:00 Anti Social Social Ride ['groupride']
2021-04-01 19:00:00+00:00 KISS Race Series ['race', 'ranked']
2021-04-01 21:00:00+00:00 Tour de Waffle ['race']
2021-04-01 23:00:00+00:00 Tempo Tester ['groupride']
2021-04-02 00:00:00+00:00 Race to the Light House ['race']
2021-04-02 01:00:00+00:00 Allez België - Oost Vlaanderen Segment Race ['race']
2021-04-02 05:00:00+00:00 The Endurance Ride ['groupride']
2021-04-02 06:00:00+00:00 Breakfast Club ['groupride']
2021-04-02 07:00:00+00:00 Sea Floor to Summit 1 ['groupride']
2021-04-02 07:00:00+00:00 Allez België - Oost Vlaanderen Segme

## Write to File

Can use to manually import into Google, other calendars

In [7]:
import tempfile, os
f = open('./rgt_events.ics', 'wb')
f.write(cal.to_ical())
f.close()

## Use gcsa for Simplified Access to Google Calendar API

* GitHub: https://github.com/kuzmoyev/google-calendar-simple-api
* Docs: https://google-calendar-simple-api.readthedocs.io/en/latest/index.html

Need to add socket timeout of 5 minutes due to slow response on my machine

### Shared Calendar

* Calendar ID: 3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com
* Google sharing link: https://calendar.google.com/calendar/u/0?cid=M2U4Z2F1OGJvbW1mamszM2o5MnJ2NWs3cTBAZ3JvdXAuY2FsZW5kYXIuZ29vZ2xlLmNvbQ
* ICS format for e.g., Apple Calendar: https://calendar.google.com/calendar/ical/3e8gau8bommfjk33j92rv5k7q0%40group.calendar.google.com/public/basic.ics


In [8]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = '3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


## Cleanup: Delete All Events from Google Calendar

Uncomment to clean up calendar.

In [9]:
#calendar.clear() # This gives an error in the Google API

#for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):
#    print('Deleting:', event, event.event_id)
#    calendar.delete_event(event)

## Find existing events, mark for update instead of creation

In [10]:
import re

reExtractName = re.compile(" \[\'.*")

df['cal_id'] = None
df['event_obj'] = None
df['found'] = False

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):

    print(event)

    rideName = reExtractName.sub("", event.summary) # Get substring from event summary with just the name

    df.loc[(df['name'] == rideName) & (df.index == event.start), ['found', 'cal_id', 'event_obj']] = [True, event.event_id, event]


2021-04-04 11:00:00+00:00 - Velotec Spring Classic #4 ['race'] 6/49.99 km/526 m
2021-04-05 17:45:00+00:00 - AHP Series #5 ['race'] 5/98.25 km/2.29 km
2021-04-08 20:00:00+00:00 - Velotec Spring Classic #5 ['race'] 3/27.52 km/265 m
2021-04-10 05:00:00+00:00 - WKG's SDW. Stage 5 ['race'] 2/14.22 km/347 m
2021-04-10 17:00:00+00:00 - WKG's SDW. Stage 5 ['race'] 7/14.22 km/347 m
2021-04-28 18:30:00+00:00 - WESTERLEY VR RACE 3.2 ['race', 'ranked'] 19/15.19 km/68 m
2021-05-12 18:30:00+00:00 - WESTERLEY VR RACE 3.3 ['race', 'ranked'] 19/15.91 km/15 m
2021-05-26 18:30:00+00:00 - WESTERLEY VR Race 3.4 ['race', 'ranked'] 20/15.65 km/64 m
2021-06-09 18:30:00+00:00 - WESTERLEY VR Race 3.5 ['race', 'ranked'] 21/15.27 km/52 m
2021-06-23 18:30:00+00:00 - WESTERLEY VR Race 3.6 ['race', 'ranked'] 19/17.74 km/39 m
2021-07-07 18:30:00+00:00 - WESTERLEY VR RACE 3.7 ['race', 'ranked'] 17/16.46 km/58 m
2021-07-21 18:30:00+00:00 - WESTERLEY VR RACE 3.8 ['race', 'ranked'] 17/15.46 km/84 m
2021-08-18 18:30:00+00

In [11]:
df.loc[df['found'] == True]


,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-04-02 05:00:00+00:00,The Endurance Ride,04-02 05:00,/events/81303,[groupride],3,38.59 km,95 m,95 m,Borrego Springs,/courses/97,False,3tdcva3vh6s42cktlj3c7m6mdg,2021-04-02 05:00:00+00:00 - The Endurance Ride...,True
2021-04-02 07:00:00+00:00,Sea Floor to Summit 1,04-02 07:00,/events/81884,[groupride],6,42.40 km,2.71 km,48 m,Hawaii_1,/courses/160791,False,3diobb3d5lp2ispr04lgknvgqc,2021-04-02 07:00:00+00:00 - Sea Floor to Summi...,True
2021-04-02 09:00:00+00:00,AHP SERIES #5,04-02 09:00,/events/81825,[race],7,98.25 km,2.29 km,2.40 km,AHP LNBR - PIOD,/courses/148480,False,fo5fhmo06t34fksagogn15e0uo,2021-04-02 09:00:00+00:00 - AHP SERIES #5 ['ra...,True
2021-04-02 17:00:00+00:00,SDW Friday Frenzy #8 ITT,04-02 17:00,/events/82961,"[race, itt, ranked]",36,19.60 km,368 m,391 m,SDW ASHDOWN FOREST TT - GS898,/courses/163531,True,1el6c4jqsjdpac7ngi8kc3l7jc,2021-04-02 17:00:00+00:00 - SDW Friday Frenzy ...,True
2021-04-03 05:00:00+00:00,WKG's SDW. Stage 4,04-03 05:00,/events/61703,[race],2,40.02 km,739 m,901 m,SDW Devils Dyke To A,/courses/108553,False,85nd77shcvf0i153t8eem3o3j4,2021-04-03 05:00:00+00:00 - WKG's SDW. Stage 4...,True
2021-04-03 07:00:00+00:00,Sea Floor to Summit 2,04-03 07:00,/events/81889,[groupride],7,41.73 km,2.41 km,93 m,Hawaii_2,/courses/160915,False,v2lnch8cqupolu1ghleo53a1mo,2021-04-03 07:00:00+00:00 - Sea Floor to Summi...,True
2021-04-03 11:00:00+00:00,WKG's SDW. Stage 4,04-03 11:00,/events/61707,[race],16,40.02 km,739 m,901 m,SDW Devils Dyke To A,/courses/108553,False,r2d3lk7t4u1ajg6d6c1uosb4f4,2021-04-03 11:00:00+00:00 - WKG's SDW. Stage 4...,True
2021-04-03 17:00:00+00:00,WKG's SDW. Stage 4,04-03 17:00,/events/61710,[race],9,40.02 km,739 m,901 m,SDW Devils Dyke To A,/courses/108553,False,t3j4kgdifo392hrrej17rtkh7o,2021-04-03 17:00:00+00:00 - WKG's SDW. Stage 4...,True
2021-04-04 07:00:00+00:00,Sea Floor to Summit 3,04-04 07:00,/events/81888,[groupride],4,42.47 km,2.65 km,133 m,Hawaii_3,/courses/160799,False,o27j64p6lokccduc5hcsrkvv7k,2021-04-04 07:00:00+00:00 - Sea Floor to Summi...,True


In [12]:
df.loc[df['found'] == False]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-04-01 16:30:00+00:00,STDistribution - Recovery,04-01 16:30,/events/84086,[groupride],20,27.35 km,161 m,161 m,STD Ride In Uster,/courses/101244,False,None,None,False
2021-04-01 16:45:00+00:00,La classique du Jeudi,04-01 16:45,/events/83671,"[race, ranked]",70,36.38 km,322 m,323 m,Critérium Lisieux,/courses/145173,True,None,None,False
2021-04-01 17:00:00+00:00,Port Talbot Wheelers TT,04-01 17:00,/events/84002,"[race, itt]",5,16.13 km,62 m,58 m,R10/22A TT Course,/courses/148804,False,None,None,False
2021-04-01 17:00:00+00:00,Tour de Waffle,04-01 17:00,/events/83292,[race],3,25.16 km,614 m,614 m,Paterberg,/courses/88,False,None,None,False
2021-04-01 18:00:00+00:00,Abandonment of Hope HC,04-01 18:00,/events/80163,[race],42,32.28 km,564 m,309 m,Abandonment Of Hope,/courses/159393,False,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-03 17:30:00+00:00,SSR San Juan Monday race,05-03 17:30,/events/84512,"[race, ranked]",1,36.96 km,373 m,282 m,San Juan De Los Terreros 1,/courses/149440,True,None,None,False
2021-05-10 10:30:00+00:00,Le Critérium du Midi #5/5,05-10 10:30,/events/84218,"[race, ranked]",3,32.19 km,427 m,427 m,Nurburgringv4,/courses/166859,True,None,None,False
2021-07-24 14:00:00+00:00,OTR WorldTour - San Sebas,07-24 14:00,/events/83872,"[race, ranked]",2,49.64 km,784 m,789 m,San Sebastian,/courses/161919,True,None,None,False


## Add All Events to Google Calendar

In [13]:
from gcsa.event import Event as gcEvent

for index, row in df.iterrows():

    evntSummary = str(str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']) + '/' + row['distance'] + '/' + row['elevationGain'])

    evntDescription = 'Signups: ' +  str(row['signUps']) + '\n' + 'Distance: ' + row['distance'] + '\n' +  'Elevation gain: ' + row['elevationGain'] + '\n' + 'Descent: ' + row['elevationLost'] + '\n' + 'https://rgtdb.com' + str(row['detailsUrl'])

    evntString = row['detailsUrl'].replace('/events/', '')

    if row['found'] == False:

        print('+New event: ', index, row['name'], row['tags'], evntString)

        evntColor = '1'

        if "groupride" in row['tags']:
            evntColor = '2'
        elif 'pro' in row['tags']:
            evntColor = '3'
        elif 'elimination' in row['tags']:
            evntColor = '4'
        elif "itt" in row['tags']:
            evntColor = '5'
        elif "race" in row['tags']:
            evntColor = '6'

        event = gcEvent(
            evntSummary,
            start=index,
            timezone='UTC',
            location=str(row['roadName']),
            description=evntDescription,
            event_id=evntString,
            color = evntColor
        )

        print('ID before add:', event.event_id)
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else:

        print('-Updating event: ', index, row['name'], row['tags'], evntString)

        event = row['event_obj']

        event.summary = evntSummary
        event.description = evntDescription

        calendar.update_event(event)

+New event:  2021-04-01 16:30:00+00:00 STDistribution - Recovery ['groupride'] 84086
ID before add: 84086
ID after add: 84086 Returned event ID: h9aupogtbdvjij0jrhcqhq5rl8
+New event:  2021-04-01 16:45:00+00:00 La classique du Jeudi ['race', 'ranked'] 83671
ID before add: 83671
ID after add: 83671 Returned event ID: p7s3okbag9vpblol4lijdnlra8
+New event:  2021-04-01 17:00:00+00:00 Port Talbot Wheelers TT ['race', 'itt'] 84002
ID before add: 84002
ID after add: 84002 Returned event ID: 3ua9ftjpqp93nuf3ts5ivradgo
+New event:  2021-04-01 17:00:00+00:00 Tour de Waffle ['race'] 83292
ID before add: 83292
ID after add: 83292 Returned event ID: lhuf8ni9tki6b510du32n5sqec
+New event:  2021-04-01 18:00:00+00:00 Abandonment of Hope HC  ['race'] 80163
ID before add: 80163
ID after add: 80163 Returned event ID: seoktiquat87bkd3vrpghld5t4
+New event:  2021-04-01 18:00:00+00:00 Tour of Flanders Group Ride with Johan Museeuw ['groupride'] 83902
ID before add: 83902
ID after add: 83902 Returned event 

In [14]:
df.loc[df['signUps'] > 9]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-04-01 16:30:00+00:00,STDistribution - Recovery,04-01 16:30,/events/84086,[groupride],20,27.35 km,161 m,161 m,STD Ride In Uster,/courses/101244,False,None,None,False
2021-04-01 16:45:00+00:00,La classique du Jeudi,04-01 16:45,/events/83671,"[race, ranked]",70,36.38 km,322 m,323 m,Critérium Lisieux,/courses/145173,True,None,None,False
2021-04-01 18:00:00+00:00,Abandonment of Hope HC,04-01 18:00,/events/80163,[race],42,32.28 km,564 m,309 m,Abandonment Of Hope,/courses/159393,False,None,None,False
2021-04-01 18:00:00+00:00,Tour of Flanders Group Ride with Johan Museeuw,04-01 18:00,/events/83902,[groupride],180,17.75 km,337 m,337 m,De Ronde,/courses/166475,False,None,None,False
2021-04-01 19:00:00+00:00,KISS Race Series,04-01 19:00,/events/82975,"[race, ranked]",22,46.30 km,114 m,114 m,Borrego Springs,/courses/97,True,None,None,False
2021-04-02 17:00:00+00:00,SDW Friday Frenzy #8 ITT,04-02 17:00,/events/82961,"[race, itt, ranked]",36,19.60 km,368 m,391 m,SDW ASHDOWN FOREST TT - GS898,/courses/163531,True,1el6c4jqsjdpac7ngi8kc3l7jc,2021-04-02 17:00:00+00:00 - SDW Friday Frenzy ...,True
2021-04-03 10:00:00+00:00,RIAK E-DUATHLON: Race 3,04-03 10:00,/events/68080,[race],13,20.70 km,106 m,106 m,Canary Wharf,/courses/72,False,None,None,False
2021-04-03 11:00:00+00:00,WKG's SDW. Stage 4,04-03 11:00,/events/61707,[race],16,40.02 km,739 m,901 m,SDW Devils Dyke To A,/courses/108553,False,r2d3lk7t4u1ajg6d6c1uosb4f4,2021-04-03 11:00:00+00:00 - WKG's SDW. Stage 4...,True
2021-04-04 11:00:00+00:00,Velotec Spring Classic #4,04-04 11:00,/events/74534,[race],10,49.99 km,526 m,526 m,Gorey_stage2_route,/courses/134818,False,52f69kimh0olgq494h3kov8hms,2021-04-04 11:00:00+00:00 - Velotec Spring Cla...,True


In [15]:
df.loc[df['name'].str.startswith('Wacky Races')]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
